In [ ]:
# input cell
# é possível adicionar um input aqui?

In [ ]:
!pip install nltk
!pip install networkx
!pip install textblob
!pip install mercury

In [ ]:
# Importa bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import mercury as mr
import re

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
sia = SentimentIntensityAnalyzer()

In [ ]:
# gerar gráfico de sentimento
def create_array_sentiment(url, resumo=False, imprime=False):
    # Faz request do artigo e extrai o texto
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    paragraphs = soup.find_all('div', attrs={"class": "curva2-5"})

    article_text = [paragraph.text for paragraph in paragraphs]
    article_text = ' '.join(article_text)

    # Resumo do texto usando algoritmo TextRank
    sentences = nltk.sent_tokenize(article_text)
    stopwords = nltk.corpus.stopwords.words('portuguese')
    word_frequencies = {}  
    for word in nltk.word_tokenize(article_text):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_scores = {}  
    for sent in sentences:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    import heapq  
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    if resumo:
        mr.Md("## Resumo:")
        mr.Md(summary)

    if imprime:
        mr.Md(f"## Sentimento: {url}")

    # Análise de sentimento frase a frase
    sentiment_values = []
    TAG_RE = re.compile('<br>')
    for sentence in sentences:
        sentence = TAG_RE.sub('', sentence)
        sentence = sentence.replace("<p>", "")
        sentence = sentence.replace("<p/>", "")
        sentence = sentence.replace("<br>", "")
        sentence = sentence.replace("<BR>", "")
        blob = TextBlob(sentence)
        sentiment = blob.sentiment.polarity
        #sentiment = sia.polarity_scores(sentence)['compound']
        if sentiment > 0:
            if imprime:
                mr.Md(f"<div style='background-color:#ccf'>{sentence} - <b>positivo</b> {sentiment}</div>")
        elif sentiment < -0:
            if imprime:
                mr.Md(f"<div style='background-color:#fcc'>{sentence} - <b>negativo</b> {sentiment}</div>")
        #else:
            #if imprime:
                # se neutro, não imprime nada, por isso, comentado
                #print(sentence, f'- neutro {sentiment}')
                #sentence = sentence
        sentiment_values.append(sentiment)
    return sentiment_values

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def chart_sentiment(url, resumo=False, imprime=False):
    # print URL analyzed
    # print(url)
    sentiment_values = create_array_sentiment(url, imprime=imprime, resumo=resumo)
    # Increase the size of the plot
    plt.figure(figsize=(20, 1))  # Adjust the width and height as desired
    # Adjust the font size of column labels
    plt.xticks(fontsize=14)
    # Create a heatmap using seaborn
    sns.heatmap([sentiment_values], cmap='coolwarm_r', annot=True, vmin=-1, vmax=1)
    # Display the heatmap
    plt.show()


# Snowglobe @ Skoob

In [ ]:
# snowglobe
chart_sentiment("https://www.skoob.com.br/livro/resenhas/985641/edicao:987022", resumo=True, imprime=True)
